In [1]:
corpus_path = '/mnt/lovit/works/fastcampus_text_ml/1st/data/corpus_10days/news/2016-10-20_article_all_normed.txt'

import sys
sys.path.append('/mnt/lovit/git/soynlp/')

In [2]:
from soynlp import DoublespaceLineCorpus

corpus = DoublespaceLineCorpus(corpus_path, iter_sent=True)
print(len(corpus))

223357


In [3]:
%%time
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_count=5)
word_extractor.train(corpus)
word_scores = word_extractor.extract()

training was done. used memory 0.691 Gbse memory 0.744 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 360721
all accessor variety was computed # words = 360721
CPU times: user 54.7 s, sys: 260 ms, total: 54.9 s
Wall time: 54.9 s


In [12]:
score

Scores(cohesion_forward=0.1943363253634125, cohesion_backward=0.07681159420289856, left_branching_entropy=3.204801502248508, right_branching_entropy=0.42721236711742844, left_accessor_variety=154, right_accessor_variety=42, leftside_frequency=4893, rightside_frequency=159)

In [14]:
for word in '연합 연합뉴 연합뉴스'.split():
    if word in word_scores:
        score = word_scores[word]
        (lav, rav) = (score.left_accessor_variety, score.right_accessor_variety)
    else:
        (lav, rav) = (0.0, 0.0)
    print('word = %s, lav = %d, rav=%d' % (word, lav, rav))

word = 연합, lav = 154, rav=42
word = 연합뉴, lav = 0, rav=0
word = 연합뉴스, lav = 138, rav=158


In [10]:
for word in '연합 연합뉴 연합뉴스'.split():
    if word in word_scores:
        score = word_scores[word]
        (lbe, rbe) = (score.left_branching_entropy, score.right_branching_entropy)
    else:
        (lbe, rbe) = (0.0, 0.0)
    print('word = {:4}, lbe = {:.3}, rbe={:.3}'.format(word, lbe, rbe))

word = 연합  , lbe = 3.2, rbe=0.427
word = 연합뉴 , lbe = 0.0, rbe=0.0
word = 연합뉴스, lbe = 3.02, rbe=3.9


In [13]:
cohesion_scores = {word:score.cohesion_forward for word, score in word_scores.items()}

In [17]:
from soynlp.tokenizer import LTokenizer
from soynlp.tokenizer import MaxScoreTokenizer
from soynlp.tokenizer import RegexTokenizer

ltokenizer = LTokenizer(scores = cohesion_scores)
ltokenizer.tokenize('아이오아이의 무대가 방송에 중계되었습니다')

['아이오아이', '의', '무대', '가', '방송', '에', '중계', '되었습니다']

In [30]:
ltokenizer.tokenize('아이오아이의 무대가 방송에 중계되었습니다', flatten=False)

[('아이오아이', '의'), ('무대', '가'), ('방송', '에'), ('중계', '되었습니다')]

In [32]:
ltokenizer.tokenize('아이오아이의 무대가 방송에 중계되었습니다', remove_r=True)

['아이오아이', '무대', '방송', '중계']

In [34]:
maxscoretokenizer = MaxScoreTokenizer(scores = cohesion_scores)
maxscoretokenizer.tokenize('아이오아이의무대가방송에중계되었습니다')

['아이오아이', '의', '무대', '가', '방송', '에', '중계', '되었습니다']

In [35]:
maxscoretokenizer.tokenize('아이오아이의무대가방송에중계되었습니다', flatten=False)

[[('아이오아이', 0, 5, 0.30063636035733476, 5),
  ('의', 5, 6, 0.0, 1),
  ('무대', 6, 8, 0.042336645588678112, 2),
  ('가', 8, 9, 0.0, 1),
  ('방송', 9, 11, 0.31949135704351284, 2),
  ('에', 11, 12, 0.0, 1),
  ('중계', 12, 14, 0.0019356503785271852, 2),
  ('되었습니다', 14, 19, 0.2762976357271788, 5)]]

In [38]:
%%time
words = [ltokenizer.tokenize(sent) for sent in corpus]

CPU times: user 16.6 s, sys: 144 ms, total: 16.8 s
Wall time: 16.7 s


In [55]:
from konlpy.tag import Twitter
twitter = Twitter()
twitter.pos('로딩시간을제외하기위한예문')

[('로딩', 'Noun'),
 ('시간', 'Noun'),
 ('을', 'Josa'),
 ('제외하기', 'Verb'),
 ('위한', 'Verb'),
 ('예문', 'Noun')]

In [56]:
%%time
twitter_words = [twitter.pos(sent) for sent in corpus]

CPU times: user 4min 29s, sys: 1.69 s, total: 4min 31s
Wall time: 4min 22s


In [59]:
twitter_words = [['%s/%s' % (w,t) for w,t in sent] for sent in twitter_words]

In [39]:
from gensim.models import Word2Vec

word2vec = Word2Vec(words)

In [67]:
print(word2vec.wv.vocab['방송'])

Vocab(count:5616, index:76, sample_int:4294967296)


In [54]:
word2vec.most_similar('방송')

[('예능프로그램', 0.6981078386306763),
 ('예능', 0.688145637512207),
 ('방영', 0.660905659198761),
 ('라디오스타', 0.6552329659461975),
 ('라디오', 0.6498782634735107),
 ('엠카운트다운', 0.6143872141838074),
 ('파워타임', 0.586585521697998),
 ('식사하셨어요', 0.5865078568458557),
 ('한끼줍쇼', 0.5841958522796631),
 ('황금어장', 0.5715700387954712)]

In [53]:
word2vec.most_similar('뉴스')

[('연예', 0.6419150233268738),
 ('화면', 0.6120390892028809),
 ('뉴스부장', 0.5935080051422119),
 ('뉴스부', 0.5910232067108154),
 ('라디오', 0.5880887508392334),
 ('채널', 0.5730870962142944),
 ('정보팀', 0.5722399950027466),
 ('속보팀', 0.5669897794723511),
 ('앵커', 0.5661150217056274),
 ('토크쇼', 0.5654663443565369)]

In [52]:
word2vec.most_similar('영화')

[('작품', 0.7261432409286499),
 ('드라마', 0.7252553701400757),
 ('흥행', 0.6903091669082642),
 ('독립영화', 0.6875913143157959),
 ('걷기왕', 0.654923677444458),
 ('감독', 0.6543536186218262),
 ('럭키', 0.6453847885131836),
 ('다큐멘터리', 0.6425734758377075),
 ('블록버스터', 0.6288458704948425),
 ('주연', 0.6285911798477173)]

In [69]:
word2vec.most_similar('아이오아이')

[('타이틀곡', 0.8425650000572205),
 ('에이핑크', 0.8301823139190674),
 ('샤이니', 0.8209962844848633),
 ('너무너무', 0.8158787488937378),
 ('다이아', 0.8083204030990601),
 ('트와이스', 0.8079981803894043),
 ('파이터', 0.8070152997970581),
 ('잠깐만', 0.7995353937149048),
 ('몬스', 0.7901242971420288),
 ('불독의', 0.7892482280731201)]

In [60]:
word2vec_twitter = Word2Vec(twitter_words)

In [63]:
word2vec_twitter.most_similar('영화/Noun')

[('드라마/Noun', 0.6787333488464355),
 ('작품/Noun', 0.677376389503479),
 ('독립영화/Noun', 0.6677168607711792),
 ('영화로/Noun', 0.6049726009368896),
 ('주연/Noun', 0.5995980501174927),
 ('럭키/Noun', 0.5985063314437866),
 ('감독/Noun', 0.5851325988769531),
 ('다큐멘터리/Noun', 0.5788881778717041),
 ('청춘/Noun', 0.5751833915710449),
 ('강동원/Noun', 0.5721665620803833)]

In [64]:
word2vec_twitter.most_similar('뉴스/Noun')

[('연합뉴스/Noun', 0.6110204458236694),
 ('연예/Noun', 0.5120304822921753),
 ('기자/Noun', 0.4866550862789154),
 ('김도연/Noun', 0.48150816559791565),
 ('렬/Noun', 0.47369953989982605),
 ('권현진/Noun', 0.47151216864585876),
 ('방지영/Noun', 0.46529433131217957),
 ('2580/Number', 0.46096765995025635),
 ('한대욱/Noun', 0.4602851867675781),
 ('김영선/Noun', 0.4595600962638855)]

In [68]:
word2vec_twitter.most_similar('방송/Noun')

[('방영/Noun', 0.6817507147789001),
 ('엠카운트다운/Noun', 0.6040224432945251),
 ('예능/Noun', 0.5873677134513855),
 ('라디오스타/Noun', 0.5550553202629089),
 ('방송사/Noun', 0.5497354865074158),
 ('첫방송/Noun', 0.5472666621208191),
 ('방송한/Verb', 0.5442488789558411),
 ('수목드라마/Noun', 0.5297601222991943),
 ('라디오/Noun', 0.5036641955375671),
 ('준영/Noun', 0.49247831106185913)]

In [70]:
word2vec_twitter.most_similar('아이오아이/Noun')

KeyError: "word '아이오아이/Noun' not in vocabulary"